In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# name the data as rets (returns)
rets = pd.read_excel('Data for coding Test.xlsx', sheet_name='problem #4')
rets.head()

,Dates,Returns,Unnamed: 2,Cumulative Returns
0,1989-12-29,0.000000,NaN,NaN
1,1990-01-01,0.000000,NaN,0.000000
2,1990-01-02,0.012922,NaN,0.012922
3,1990-01-03,-0.001654,NaN,0.011268
4,1990-01-04,-0.003838,NaN,0.007430


In [3]:
rets = rets[['Dates', 'Returns']] #take only the two columns
rets.head()

,Dates,Returns
0,1989-12-29,0.000000
1,1990-01-01,0.000000
2,1990-01-02,0.012922
3,1990-01-03,-0.001654
4,1990-01-04,-0.003838


In [4]:
rets['Year'] = pd.DatetimeIndex(rets['Dates']).year
rets.head()

,Dates,Returns,Year
0,1989-12-29,0.000000,1989
1,1990-01-01,0.000000,1990
2,1990-01-02,0.012922,1990
3,1990-01-03,-0.001654,1990
4,1990-01-04,-0.003838,1990


In [5]:
rets.dtypes

Dates      datetime64[ns]
Returns           float64
Year                int64
dtype: object

In [6]:
# take the first year of data, as an example, temp is not a good name but this is just walking through the methodology, the section after this will have better names
sta_1990 = rets.loc[rets["Year"] == 1990]
sta_1990

,Dates,Returns,Year
1,1990-01-01,0.000000,1990
2,1990-01-02,0.012922,1990
3,1990-01-03,-0.001654,1990
4,1990-01-04,-0.003838,1990
5,1990-01-05,-0.010907,1990
...,...,...,...
257,1990-12-25,-0.000009,1990
258,1990-12-26,-0.000713,1990
259,1990-12-27,0.002362,1990
260,1990-12-28,0.001133,1990


In [7]:
import math

In [8]:
# STATIC METHOD
vol = 0.1 # assumption
s_interval = [-2,-1,0,1,2]
oldS = [0.1, 0.1, 0.1, 0.6, 0.1] # Static assumption that I made on the initial states to make Sharpe to be 0.5
def get_newS(annualData, s_interval, oldS):
    res = []
    for i in range(len(oldS)):
        sigma = vol / math.sqrt(len(annualData))
        mu = vol * s_interval[i] / len(annualData)
        sumSq = ((annualData - mu)**2).sum()
        # (1/(math.sqrt(2*math.pi)*sigma))**len(annualData) part is thrown away because we are going to scale it 
        r_givenS = math.exp((-1/(2*sigma**2))*sumSq)
        new = r_givenS * oldS[i]
        res.append(new)
    total = sum(res)
    result = []
    for s in res:
        s /= total
        result.append(s)
    return result

In [9]:
annualData1990 = sta_1990["Returns"]
newS = get_newS(annualData1990, s_interval, oldS)
newS # the year-end updated belief

[0.2451451647360552,
 0.35137369380694056,
 0.18527664370085473,
 0.2156397769767778,
 0.0025647207793718227]

In [10]:
# calculate the updated Sharpe associated with the new belief
new_Sharpe = sum([newS[i]*s_interval[i] for i in range(len(newS))])
new_Sharpe # -0.62089, therefore is a money loser going forward, thus the strategy should be retired

-0.6208948047435294

In [11]:
# Below is the DYNAMIC METHOD
dy_1990 = rets.loc[rets["Year"] == 1990]
dy_1990

,Dates,Returns,Year
1,1990-01-01,0.000000,1990
2,1990-01-02,0.012922,1990
3,1990-01-03,-0.001654,1990
4,1990-01-04,-0.003838,1990
5,1990-01-05,-0.010907,1990
...,...,...,...
257,1990-12-25,-0.000009,1990
258,1990-12-26,-0.000713,1990
259,1990-12-27,0.002362,1990
260,1990-12-28,0.001133,1990


In [12]:
# S1 is stable at 0.5 forever
dy_1990['S1'] = 0.5
dy_1990

,Dates,Returns,Year,S1
1,1990-01-01,0.000000,1990,0.5
2,1990-01-02,0.012922,1990,0.5
3,1990-01-03,-0.001654,1990,0.5
4,1990-01-04,-0.003838,1990,0.5
5,1990-01-05,-0.010907,1990,0.5
...,...,...,...,...
257,1990-12-25,-0.000009,1990,0.5
258,1990-12-26,-0.000713,1990,0.5
259,1990-12-27,0.002362,1990,0.5
260,1990-12-28,0.001133,1990,0.5


In [13]:
# S2 is 0.5 for half a year and then decays to 0 abruptly
s2 = [0.5]*130 + [0]*131
dy_1990["S2"] = s2
dy_1990

,Dates,Returns,Year,S1,S2
1,1990-01-01,0.000000,1990,0.5,0.5
2,1990-01-02,0.012922,1990,0.5,0.5
3,1990-01-03,-0.001654,1990,0.5,0.5
4,1990-01-04,-0.003838,1990,0.5,0.5
5,1990-01-05,-0.010907,1990,0.5,0.5
...,...,...,...,...,...
257,1990-12-25,-0.000009,1990,0.5,0.0
258,1990-12-26,-0.000713,1990,0.5,0.0
259,1990-12-27,0.002362,1990,0.5,0.0
260,1990-12-28,0.001133,1990,0.5,0.0


In [14]:
# S3 decays exponentially to 0 from 0.5 at a rate of 0.01 per day
s3 = [0.5*(1-0.01)**i for i in range(261)] # 261 is a magic number, which is the length of the priod.
dy_1990["S3"] = s3
dy_1990

,Dates,Returns,Year,S1,S2,S3
1,1990-01-01,0.000000,1990,0.5,0.5,0.500000
2,1990-01-02,0.012922,1990,0.5,0.5,0.495000
3,1990-01-03,-0.001654,1990,0.5,0.5,0.490050
4,1990-01-04,-0.003838,1990,0.5,0.5,0.485150
5,1990-01-05,-0.010907,1990,0.5,0.5,0.480298
...,...,...,...,...,...,...
257,1990-12-25,-0.000009,1990,0.5,0.0,0.038157
258,1990-12-26,-0.000713,1990,0.5,0.0,0.037776
259,1990-12-27,0.002362,1990,0.5,0.0,0.037398
260,1990-12-28,0.001133,1990,0.5,0.0,0.037024


In [15]:
# S4 decays immediately to 0
dy_1990["S4"] = 0
dy_1990

,Dates,Returns,Year,S1,S2,S3,S4
1,1990-01-01,0.000000,1990,0.5,0.5,0.500000,0
2,1990-01-02,0.012922,1990,0.5,0.5,0.495000,0
3,1990-01-03,-0.001654,1990,0.5,0.5,0.490050,0
4,1990-01-04,-0.003838,1990,0.5,0.5,0.485150,0
5,1990-01-05,-0.010907,1990,0.5,0.5,0.480298,0
...,...,...,...,...,...,...,...
257,1990-12-25,-0.000009,1990,0.5,0.0,0.038157,0
258,1990-12-26,-0.000713,1990,0.5,0.0,0.037776,0
259,1990-12-27,0.002362,1990,0.5,0.0,0.037398,0
260,1990-12-28,0.001133,1990,0.5,0.0,0.037024,0


In [16]:
vol = 0.1 # assumption
oldS = [0.5, 0.1, 0.25, 0.15] # dynamic assumption same as the paper for the 4 types of strategies to make Sharpe 0.5
def get_newS(annualData, oldS):
    res = []
    for i in range(len(oldS)): # modify the first three strategies allocation because the data mining is static
        sigma = vol / math.sqrt(len(annualData))
        colName = "S" + str(i+1)
        annualData["mu"] = annualData[colName] * vol / len(annualData)
        sumSq = ((annualData["Returns"] - annualData["mu"])**2).sum()
        # (1/(math.sqrt(2*math.pi)*sigma))**len(annualData) part is thrown away because we are going to scale it 
        r_givenS = math.exp((-1/(2*sigma**2))*sumSq)
        new = r_givenS * oldS[i]
        res.append(new) 
    total = sum(res)
    result = []
    for s in res:
        s /= total
        result.append(s)
    return result

In [17]:
newS = get_newS(dy_1990, oldS)
newS # the year-end updated belief

[0.4021488354729455,
 0.08348388221050947,
 0.27263047301751636,
 0.24173680929902872]

In [18]:
# calculate the updated Sharpe associated with the new belief
newSharpe = dy_1990.iloc[-1]["S1"]*newS[0] + dy_1990.iloc[-1]["S2"]*newS[1] + dy_1990.iloc[-1]["S3"]*newS[2] + dy_1990.iloc[-1]["S3"]*newS[3]
newSharpe # 0.22, which could be a signal to retire the strategy

0.21992800242273372

In [19]:
# But let's say that 0.22 is still a GO for the strategy, and we then study a stream-lined version of this procedure going forward
# But first, set columns for the dynamic sharpe ratios
rets['S1'] = 0.5 
s2 = [0.5]*130 + [0]*(len(rets)-130) # s2 is decayed down to 0 after 0.5 year
rets["S2"] = s2
s3 = [0.5*(1-0.01)**i for i in range(len(rets))] 
rets["S3"] = s3
rets["S4"] = 0
rets

,Dates,Returns,Year,S1,S2,S3,S4
0,1989-12-29,0.000000,1989,0.5,0.5,5.000000e-01,0
1,1990-01-01,0.000000,1990,0.5,0.5,4.950000e-01,0
2,1990-01-02,0.012922,1990,0.5,0.5,4.900500e-01,0
3,1990-01-03,-0.001654,1990,0.5,0.5,4.851495e-01,0
4,1990-01-04,-0.003838,1990,0.5,0.5,4.802980e-01,0
...,...,...,...,...,...,...,...
7738,2019-08-28,-0.001241,2019,0.5,0.0,8.396645e-35,0
7739,2019-08-29,0.008387,2019,0.5,0.0,8.312679e-35,0
7740,2019-08-30,0.003461,2019,0.5,0.0,8.229552e-35,0
7741,2019-09-02,0.002711,2019,0.5,0.0,8.147256e-35,0


In [20]:
# study 1991 and onwards
def get_Sharpe_list(newS):
    res = []
    for year in range(1991,2019): # exclude year 2019 because the data is not complete
        tempD = rets.loc[rets["Year"] == year]
        newS = get_newS(tempD, newS) # keep updating newS
        newSharpe = tempD.iloc[-1]["S1"]*newS[0] + tempD.iloc[-1]["S2"]*newS[1] + tempD.iloc[-1]["S3"]*newS[2] + tempD.iloc[-1]["S3"]*newS[3]
        res.append(newSharpe)
    return res

In [21]:
# grab the 1990 year-end belief as a starting point, return the list of the sharpe ratios of year 1991 to 2018
get_Sharpe_list(newS)

[0.12504363331704033,
 0.08883844833513084,
 0.12732138999103576,
 0.054940625493992416,
 0.09192420736334568,
 0.15707096935571446,
 0.27393581010458484,
 0.2582177458490865,
 0.28037069166212586,
 0.25926147702398283,
 0.3302863843051052,
 0.3849317738573388,
 0.41198671630707884,
 0.4249911408055196,
 0.4575924530819915,
 0.47289811370510076,
 0.466213103546399,
 0.4875203510169181,
 0.4891164671420012,
 0.48601289703478223,
 0.47945643325212706,
 0.4719846342812984,
 0.4875443914111177,
 0.4826092613608695,
 0.4753616304508928,
 0.478865478478587,
 0.48961458037999683,
 0.4817155837852983]

In [22]:
# Comment on the above list of sharpe ratios
# From the above list of sharpe ratios, we can tell that the strategy works well after a couple of years, but the question is if the strategy could survice after the first 5 years of decreasing performance.